In [1]:
import pandas as pd
import gensim

In [2]:
train = pd.read_csv('X_train.csv', encoding = 'cp949')
test = pd.read_csv('X_test.csv', encoding = 'cp949')
y_train = pd.read_csv('y_train.csv', encoding = 'cp949')  

In [3]:
data = pd.concat([train, test]).reset_index(drop=True)

## corner_nm

In [4]:
%%writefile word2vec_corner.py

### Imports
import pandas as pd
import numpy as np
import os


### Read data
train = pd.read_csv('X_train.csv', encoding='cp949')
test = pd.read_csv('X_test.csv', encoding='cp949')


### Make corpus
p_level = 'corner_nm'  # 상품 분류 수준

# W2V 학습데이터가 부족하여 구매한 상품 목록으로부터 n배 oversampling을 수행
def oversample(x, n, seed=0):
    if n == 0:
        return list(x)
    uw = np.unique(x)
    bs = np.array([])
    np.random.seed(seed)
    for j in range(n):
        bs = np.append(bs, np.random.choice(uw, len(uw), replace=True))  # 복원추출
    return list(bs)

train_corpus = list(train.groupby('custid')[p_level].agg(oversample, 20))
test_corpus = list(test.groupby('custid')[p_level].agg(oversample, 20))


### Training the Word2Vec model
num_features = 100 # 단어 벡터 차원 수
min_word_count = 1 # 최소 단어 수
context = 5 # 학습 윈도우(인접한 단어 리스트) 크기

# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
w2v = word2vec.Word2Vec(train_corpus, 
                        vector_size=num_features, 
                        min_count=min_word_count,
                        window=context,
                        seed=0, workers=1)
# 필요없는 메모리 unload
w2v.init_sims(replace=True)


### Make features
# 구매상품에 해당하는 벡터의 평균/최소/최대 벡터를 feature로 만드는 전처리기
class EmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = num_features
    def fit(self, X):
        return self
    def transform(self, X):
        return np.array([
            np.hstack([
                np.max([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.min([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),                
                #np.std([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0)                
            ]) 
            for words in X
        ]) 

# W2V 기반 feature 생성
train_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).fit(train_corpus).transform(train_corpus))
test_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).transform(test_corpus))

train_features.columns = ['v'+f'{c+1:03d}' for c in train_features.columns]
test_features.columns = ['v'+f'{c+1:03d}' for c in test_features.columns]

# 학습용과 제출용 데이터로 분리
X_train_corner = pd.concat([pd.DataFrame({'custid': np.sort(train['custid'].unique())}), train_features], axis=1)#.to_csv('X_train_buyer.csv', index=False)
X_test_corner = pd.concat([pd.DataFrame({'custid': np.sort(test['custid'].unique())}), test_features], axis=1)#.to_csv('X_test_buyer.csv', index=False)

Overwriting word2vec_corner.py


In [5]:
%run word2vec_corner.py

C:\Users\hwangtaegyun\Desktop\머신러닝\kml2022s\word2vec_corner.py:46: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


## brd_nm

In [6]:
%%writefile word2vec_brd.py

### Imports
import pandas as pd
import numpy as np
import os


### Read data
train = pd.read_csv('X_train.csv', encoding='cp949')
test = pd.read_csv('X_test.csv', encoding='cp949')


### Make corpus
p_level = 'brd_nm'  # 상품 분류 수준

# W2V 학습데이터가 부족하여 구매한 상품 목록으로부터 n배 oversampling을 수행
def oversample(x, n, seed=0):
    if n == 0:
        return list(x)
    uw = np.unique(x)
    bs = np.array([])
    np.random.seed(seed)
    for j in range(n):
        bs = np.append(bs, np.random.choice(uw, len(uw), replace=True))
    return list(bs)

train_corpus = list(train.groupby('custid')[p_level].agg(oversample, 20))
test_corpus = list(test.groupby('custid')[p_level].agg(oversample, 20))


### Training the Word2Vec model
num_features = 300 # 단어 벡터 차원 수
min_word_count = 1 # 최소 단어 수
context = 5 # 학습 윈도우(인접한 단어 리스트) 크기

# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
w2v = word2vec.Word2Vec(train_corpus, 
                        vector_size=num_features, 
                        min_count=min_word_count,
                        window=context,
                        seed=0, workers=1)
# 필요없는 메모리 unload
w2v.init_sims(replace=True)


### Make features
# 구매상품에 해당하는 벡터의 평균/최소/최대 벡터를 feature로 만드는 전처리기
class EmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = num_features
    def fit(self, X):
        return self
    def transform(self, X):
        return np.array([
            np.hstack([
                np.max([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.min([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),                
                #np.std([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0)                
            ]) 
            for words in X
        ]) 

# W2V 기반 feature 생성
train_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).fit(train_corpus).transform(train_corpus))
test_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).transform(test_corpus))

train_features.columns = ['v'+f'{c+1:03d}' for c in train_features.columns]
test_features.columns = ['v'+f'{c+1:03d}' for c in test_features.columns]

# 학습용과 제출용 데이터로 분리
X_train_brd = pd.concat([pd.DataFrame({'custid': np.sort(train['custid'].unique())}), train_features], axis=1)#.to_csv('X_train_buyer.csv', index=False)
X_test_brd = pd.concat([pd.DataFrame({'custid': np.sort(test['custid'].unique())}), test_features], axis=1)#.to_csv('X_test_buyer.csv', index=False)

Overwriting word2vec_brd.py


In [7]:
%run word2vec_brd.py

C:\Users\hwangtaegyun\Desktop\머신러닝\kml2022s\word2vec_brd.py:46: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


## pc_nm

In [8]:
%%writefile word2vec_pc.py

### Imports
import pandas as pd
import numpy as np
import os


### Read data
train = pd.read_csv('X_train.csv', encoding='cp949')
test = pd.read_csv('X_test.csv', encoding='cp949')


### Make corpus
p_level = 'pc_nm'  # 상품 분류 수준

# W2V 학습데이터가 부족하여 구매한 상품 목록으로부터 n배 oversampling을 수행
def oversample(x, n, seed=0):
    if n == 0:
        return list(x)
    uw = np.unique(x)
    bs = np.array([])
    np.random.seed(seed)
    for j in range(n):
        bs = np.append(bs, np.random.choice(uw, len(uw), replace=True))
    return list(bs)

train_corpus = list(train.groupby('custid')[p_level].agg(oversample, 20))
test_corpus = list(test.groupby('custid')[p_level].agg(oversample, 20))


### Training the Word2Vec model
num_features = 50 # 단어 벡터 차원 수
min_word_count = 1 # 최소 단어 수
context = 5 # 학습 윈도우(인접한 단어 리스트) 크기

# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
w2v = word2vec.Word2Vec(train_corpus, 
                        vector_size=num_features, 
                        min_count=min_word_count,
                        window=context,
                        seed=0, workers=1)
# 필요없는 메모리 unload
w2v.init_sims(replace=True)


### Make features
# 구매상품에 해당하는 벡터의 평균/최소/최대 벡터를 feature로 만드는 전처리기
class EmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = num_features
    def fit(self, X):
        return self
    def transform(self, X):
        return np.array([
            np.hstack([
                np.max([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.min([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),                
                #np.std([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0)                
            ]) 
            for words in X
        ]) 

# W2V 기반 feature 생성
train_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).fit(train_corpus).transform(train_corpus))
test_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).transform(test_corpus))

train_features.columns = ['v'+f'{c+1:03d}' for c in train_features.columns]
test_features.columns = ['v'+f'{c+1:03d}' for c in test_features.columns]

# 학습용과 제출용 데이터로 분리
X_train_pc = pd.concat([pd.DataFrame({'custid': np.sort(train['custid'].unique())}), train_features], axis=1)#.to_csv('X_train_buyer.csv', index=False)
X_test_pc = pd.concat([pd.DataFrame({'custid': np.sort(test['custid'].unique())}), test_features], axis=1)#.to_csv('X_test_buyer.csv', index=False)

Overwriting word2vec_pc.py


In [9]:
%run word2vec_pc.py

C:\Users\hwangtaegyun\Desktop\머신러닝\kml2022s\word2vec_pc.py:46: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


## part_nm

In [10]:
%%writefile word2vec_part.py

### Imports
import pandas as pd
import numpy as np
import os


### Read data
train = pd.read_csv('X_train.csv', encoding='cp949')
test = pd.read_csv('X_test.csv', encoding='cp949')


### Make corpus
p_level = 'part_nm'  # 상품 분류 수준

# W2V 학습데이터가 부족하여 구매한 상품 목록으로부터 n배 oversampling을 수행
def oversample(x, n, seed=0):
    if n == 0:
        return list(x)
    uw = np.unique(x)
    bs = np.array([])
    np.random.seed(seed)
    for j in range(n):
        bs = np.append(bs, np.random.choice(uw, len(uw), replace=True))
    return list(bs)

train_corpus = list(train.groupby('custid')[p_level].agg(oversample, 20))
test_corpus = list(test.groupby('custid')[p_level].agg(oversample, 20))


### Training the Word2Vec model
num_features = 100 # 단어 벡터 차원 수
min_word_count = 1 # 최소 단어 수
context = 5 # 학습 윈도우(인접한 단어 리스트) 크기

# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
w2v = word2vec.Word2Vec(train_corpus, 
                        vector_size=num_features, 
                        min_count=min_word_count,
                        window=context,
                        seed=0, workers=1)
# 필요없는 메모리 unload
w2v.init_sims(replace=True)


### Make features
# 구매상품에 해당하는 벡터의 평균/최소/최대 벡터를 feature로 만드는 전처리기
class EmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = num_features
    def fit(self, X):
        return self
    def transform(self, X):
        return np.array([
            np.hstack([
                np.max([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.min([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),                
                #np.std([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0)                
            ]) 
            for words in X
        ]) 

# W2V 기반 feature 생성
train_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).fit(train_corpus).transform(train_corpus))
test_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).transform(test_corpus))

train_features.columns = ['v'+f'{c+1:03d}' for c in train_features.columns]
test_features.columns = ['v'+f'{c+1:03d}' for c in test_features.columns]

# 학습용과 제출용 데이터로 분리
X_train_part = pd.concat([pd.DataFrame({'custid': np.sort(train['custid'].unique())}), train_features], axis=1)#.to_csv('X_train_buyer.csv', index=False)
X_test_part = pd.concat([pd.DataFrame({'custid': np.sort(test['custid'].unique())}), test_features], axis=1)#.to_csv('X_test_buyer.csv', index=False)

Overwriting word2vec_part.py


In [11]:
%run word2vec_part.py

C:\Users\hwangtaegyun\Desktop\머신러닝\kml2022s\word2vec_part.py:46: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


## customer_info

In [12]:
%%writefile word2vec_customer_info.py

### Imports
import pandas as pd
import numpy as np
import os


### Read data
train = pd.read_csv('X_train.csv', encoding='cp949')
test = pd.read_csv('X_test.csv', encoding='cp949')
train['customer_info'] = train['brd_nm'].astype(str) + '_' + train['corner_nm'].astype(str) + '_' + train['pc_nm'].astype(str) + '_' + train['part_nm'].astype(str) + '_' + train['str_nm'].astype(str) + '_' + train['team_nm'].astype(str) + '_' + train['buyer_nm'].astype(str)
test['customer_info'] = test['brd_nm'].astype(str) + '_' + test['corner_nm'].astype(str) + '_' + test['pc_nm'].astype(str) + '_' + test['part_nm'].astype(str) + '_' + test['str_nm'].astype(str) + '_' + test['team_nm'].astype(str) + '_' + test['buyer_nm'].astype(str)

### Make corpus
p_level = 'customer_info'  # 상품 분류 수준

# W2V 학습데이터가 부족하여 구매한 상품 목록으로부터 n배 oversampling을 수행
def oversample(x, n, seed=0):
    if n == 0:
        return list(x)
    uw = np.unique(x)
    bs = np.array([])
    np.random.seed(seed)
    for j in range(n):
        bs = np.append(bs, np.random.choice(uw, len(uw), replace=True))
    return list(bs)

train_corpus = list(train.groupby('custid')[p_level].agg(oversample, 20))
test_corpus = list(test.groupby('custid')[p_level].agg(oversample, 20))


### Training the Word2Vec model
num_features = 100 # 단어 벡터 차원 수
min_word_count = 1 # 최소 단어 수
context = 5 # 학습 윈도우(인접한 단어 리스트) 크기

# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
w2v = word2vec.Word2Vec(train_corpus, 
                        vector_size=num_features, 
                        min_count=min_word_count,
                        window=context,
                        seed=0, workers=1)
# 필요없는 메모리 unload
w2v.init_sims(replace=True)


### Make features
# 구매상품에 해당하는 벡터의 평균/최소/최대 벡터를 feature로 만드는 전처리기
class EmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = num_features
    def fit(self, X):
        return self
    def transform(self, X):
        return np.array([
            np.hstack([
                np.max([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.min([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),                
                #np.std([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0)                
            ]) 
            for words in X
        ]) 

# W2V 기반 feature 생성
train_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).fit(train_corpus).transform(train_corpus))
test_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).transform(test_corpus))

train_features.columns = ['v'+f'{c+1:03d}' for c in train_features.columns]
test_features.columns = ['v'+f'{c+1:03d}' for c in test_features.columns]

# 학습용과 제출용 데이터로 분리
X_train_customer_info = pd.concat([pd.DataFrame({'custid': np.sort(train['custid'].unique())}), train_features], axis=1)#.to_csv('X_train_buyer.csv', index=False)
X_test_customer_info = pd.concat([pd.DataFrame({'custid': np.sort(test['custid'].unique())}), test_features], axis=1)#.to_csv('X_test_buyer.csv', index=False)

Overwriting word2vec_customer_info.py


In [13]:
%run word2vec_customer_info.py

C:\Users\hwangtaegyun\Desktop\머신러닝\kml2022s\word2vec_customer_info.py:47: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


In [14]:
train['custid'].nunique()

21587

In [15]:
X_train_corner

,custid,v001,v002,v003,v004,v005,v006,v007,v008,v009,...,v291,v292,v293,v294,v295,v296,v297,v298,v299,v300
0,0,0.056887,0.204296,0.149636,0.197877,0.011895,0.125749,0.243023,0.012182,0.061884,...,-0.033926,-0.020497,-0.026160,-0.110019,-0.010025,-0.055391,-0.079253,0.150992,-0.084426,0.038927
1,2,0.191458,0.204296,0.233395,0.197877,0.040383,0.156419,0.243023,0.012182,0.016669,...,0.009210,0.018467,-0.023277,-0.031402,-0.038590,0.004241,-0.114791,0.043011,-0.039728,0.051065
2,3,0.165631,0.204296,0.242314,0.197877,0.168026,0.224666,0.243023,0.105736,0.279617,...,0.028608,0.013825,-0.014507,-0.065977,-0.047070,0.005692,-0.039338,0.028312,-0.037684,-0.010549
3,4,0.101547,0.204296,-0.021308,0.197877,0.126062,0.071227,0.243023,0.012068,-0.007040,...,0.002926,0.128726,0.003520,0.034850,0.042530,-0.022697,0.036814,-0.005544,0.004125,0.093528
4,5,0.191458,0.204296,0.233395,0.197877,0.186105,0.204068,0.243023,0.169524,0.229982,...,0.002585,0.024336,0.010207,-0.024842,-0.080310,0.013839,-0.019896,0.048257,-0.027887,0.004168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,29995,0.096043,0.204296,0.178591,0.226137,0.186105,0.204068,0.243023,0.180266,0.116643,...,0.057960,0.021938,-0.068603,0.019180,0.013408,-0.006916,-0.018961,0.062378,-0.078828,0.003307
21583,29996,0.144552,0.172708,0.010097,0.184141,0.108564,0.185031,0.184655,0.069782,0.122507,...,0.011448,-0.005515,-0.071838,0.024338,0.061403,0.081722,0.018449,-0.031068,-0.051589,0.008815
21584,29997,0.106106,0.204296,0.155505,0.197877,0.149265,0.086596,0.243023,0.169524,0.153283,...,0.035744,0.012318,-0.030162,-0.000588,-0.013196,-0.036134,-0.010156,0.011357,0.023214,-0.000401
21585,29998,0.148576,0.204296,-0.027938,0.197877,0.106893,0.125749,0.243023,0.045363,-0.009854,...,0.025380,0.005048,-0.008971,-0.086903,-0.014310,0.053836,-0.059484,-0.036537,-0.063727,0.015486


# Feature Merge

In [16]:
del X_train_corner['custid']
del X_test_corner['custid']
del X_train_brd['custid']
del X_test_brd['custid']
del X_train_pc['custid']
del X_test_pc['custid']
del X_train_part['custid']
del X_test_part['custid']
del X_train_customer_info['custid']
del X_test_customer_info['custid']

In [17]:
X_train_corner.columns = X_train_corner.columns.map(lambda x : "corner_" + str(x))
X_test_corner.columns = X_test_corner.columns.map(lambda x : "corner_" + str(x))
X_train_brd.columns = X_train_brd.columns.map(lambda x : "brd_" + str(x))
X_test_brd.columns = X_test_brd.columns.map(lambda x : "brd_" + str(x))
X_train_pc.columns = X_train_pc.columns.map(lambda x : "pc_" + str(x))
X_test_pc.columns = X_test_pc.columns.map(lambda x : "pc_" + str(x))
X_train_part.columns = X_train_part.columns.map(lambda x : "part_" + str(x))
X_test_part.columns = X_test_part.columns.map(lambda x : "part_" + str(x))
X_train_customer_info.columns = X_train_customer_info.columns.map(lambda x : "customer_info_" + str(x))
X_test_customer_info.columns = X_test_customer_info.columns.map(lambda x : "customer_info_" + str(x))

In [20]:
w2v_features_train = pd.concat([X_train_corner, X_train_brd, X_train_pc, X_train_part, X_train_customer_info], axis=1) ; w2v_features_train
w2v_features_test = pd.concat([X_test_corner, X_test_brd, X_test_pc, X_test_part, X_test_customer_info], axis=1) ; w2v_features_test

,corner_v001,corner_v002,corner_v003,corner_v004,corner_v005,corner_v006,corner_v007,corner_v008,corner_v009,corner_v010,...,customer_info_v291,customer_info_v292,customer_info_v293,customer_info_v294,customer_info_v295,customer_info_v296,customer_info_v297,customer_info_v298,customer_info_v299,customer_info_v300
0,0.168851,0.204296,0.299397,0.216977,0.138331,0.224666,0.243023,0.180266,0.180750,0.185686,...,-0.037653,-0.001484,-0.020813,-0.055448,0.017109,-0.006111,0.055517,0.009039,-0.012030,0.072338
1,0.146209,0.204296,0.242314,0.197877,0.186105,0.204068,0.243023,0.180266,0.163607,0.103400,...,-0.029030,0.010975,-0.014133,-0.060091,-0.038459,0.017820,0.006929,0.016060,-0.016071,0.006643
2,0.137170,0.204296,0.132994,0.197877,0.188748,0.109339,0.243023,0.182364,0.196390,0.133698,...,0.011038,-0.091655,-0.007590,-0.009337,0.020900,-0.007950,0.047982,-0.000275,-0.034780,0.027391
3,-0.039974,-0.054913,0.010097,0.184141,0.083852,-0.082661,0.039018,-0.000735,-0.036910,-0.126528,...,-0.003080,-0.127154,-0.027601,-0.010497,0.022230,0.042322,0.093091,-0.098226,-0.106143,0.113597
4,-0.047150,0.204296,-0.008322,0.197877,0.042577,-0.003618,0.243023,0.012068,0.088701,0.117397,...,0.025820,-0.056540,0.014996,0.003111,-0.015839,-0.047980,-0.028539,0.035711,0.004489,0.040221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,0.221683,0.204296,0.106887,0.197877,0.005162,-0.003618,0.243023,0.012068,0.183859,0.148194,...,0.010272,0.102751,-0.155536,0.047139,-0.024476,0.066521,0.092263,0.037041,0.009806,-0.007837
14376,-0.047150,0.204296,-0.122241,0.197877,-0.040419,-0.003618,0.243023,0.012068,-0.036919,-0.134704,...,0.108693,-0.118788,-0.002599,-0.051196,-0.111106,0.010698,0.094173,-0.016554,0.038679,0.060349
14377,-0.047150,0.204296,0.109177,0.226137,-0.022702,0.048333,0.243023,0.076473,-0.014092,0.029511,...,0.000036,0.099930,-0.043911,0.016829,-0.066134,0.069977,-0.024249,0.078696,0.060989,0.000194
14378,0.144552,0.172708,0.072305,0.135013,0.014147,0.098543,0.146069,0.203590,0.140965,0.148194,...,0.001012,-0.077672,-0.086397,-0.069124,0.078886,0.092147,0.018202,-0.046397,0.066673,0.096268


In [21]:
w2v_features_train.to_csv('hwang_w2v_features_train.csv', index=False)
w2v_features_test.to_csv('hwang_w2v_features_test.csv', index=False)